# Finding prices using LLMs and Langgraph

The objective on this notebook is to find a price from some object from specific URL's, in the notebook example, we will use the [zoom](https://www.zoom.com.br).

For this task, we gonna use Langgraph and Langchain, to divide this task in some particular tasks. 

---

Importing packages:

In [26]:
from dotenv import load_dotenv

from langchain_ollama.chat_models import ChatOllama

from langchain_community.document_loaders import WebBaseLoader

from langchain_core.messages import (
    AnyMessage, 
    HumanMessage, 
    SystemMessage
)

from re import search

from os import getenv

Configurations:

In [27]:
load_dotenv()

True

In [28]:
OLLAMA_URL = getenv("OLLAMA_URL")
LLM_MODEL = getenv("LLM_MODEL")

In [29]:
RETAIL_URL = "https://www.zoom.com.br"

---

First, we need to define a way to use the retail store url to search some prices!

In [30]:
RETAIL_SUFIX = lambda input: f"{RETAIL_URL}/search?q={input.replace(' ','%20')}"

Now, let's consume the first retail for some respective input. For this, we will use 

In [31]:
INPUT = "ar condicionado"

In [32]:
url_search = RETAIL_SUFIX(INPUT)

In [33]:
url_search

'https://www.zoom.com.br/search?q=ar%20condicionado'

In [34]:
web_loaded = WebBaseLoader(url_search)

In [35]:
[document] = web_loaded.load()

Let's dive in on what is on the document!

In [36]:
document.dict().keys()

/tmp/ipykernel_32304/1084269162.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  document.dict().keys()


dict_keys(['id', 'metadata', 'page_content', 'type'])

In [37]:
document.id

In [38]:
document.metadata

{'source': 'https://www.zoom.com.br/search?q=ar%20condicionado',
 'title': 'Ar condicionado com o menor preço | Zoom ',
 'description': 'Buscando o menor preço para ar condicionado LG, Gree, Philco e mais? No Zoom você compara as melhores ofertas com segurança e ainda aproveita cupons e cashback.',
 'language': 'pt-BR'}

In [39]:
print(document.page_content)

Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.000 

In [40]:
document.type

'Document'

Now, we can use a textual content in a LLM!

But before, let's find a way to delimiter the retails on the search!

In [41]:
content_split_lines = document.page_content.splitlines()

In [42]:
RETAILS = [
    "Casas Bahia",
    "Extra",
    "Magazine Luiza"   
]

In [43]:
content_split_lines

['Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.00

In [44]:
pattern = rf'({"|".join(RETAILS)})'

pattern

'(Casas Bahia|Extra|Magazine Luiza)'

In [45]:
lines_with_retails = [content.strip() for content in content_split_lines if search(pattern, content)]

In [46]:
lines_with_retails

['Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.00

Now, let's send this to a LLM!

In [47]:
chat = ChatOllama(base_url=OLLAMA_URL, model=LLM_MODEL)

In [48]:
breakline = lambda: '\n'

messages: list[AnyMessage] = [
    SystemMessage(content="Você é um assistente especializado em ler uma série de ofertas com diferentes preços e trazer a melhor oferta e a sua respectiva loja!", name="System"),
    HumanMessage(content=f"Me dê a melhor oferta a partir das seguintes opções:\n{breakline().join(lines_with_retails)}\nNão esqueça de me falar onde comprar", name="Author")
]

In [49]:
answer = chat.invoke(messages)

In [50]:
print(answer.content)

Olá! Estou aqui para ajudar.

Após analisar as ofertas disponíveis, eu recomendo o seguinte produto:

**Ar-Condicionado Split Hi Wall LG Dual Inverter Voice 12000 BTUs Frio Inverter S4-Q12JA314**

Preço: R$ 2.374,05

Loja: Extra

Você pode comprar este produto na loja Extra ou no site de compras online da Amazon.

Além disso, com o uso do aplicativo Zoom e aproveitar as ofertas especiais disponíveis, você também pode ganhar cashback e economizar mais dinheiro ao fazer a compra.

Aqui estão as informações das ofertas que eu considerei:

* Ar-Condicionado Split Hi Wall LG Dual Inverter Voice 12000 BTUs Frio Inverter S4-Q12JA314: R$ 2.374,05 na loja Extra
* Ar-Condicionado Portátil Philco 12000 BTUs Frio PAC12000F54: R$ 2.572,11 na loja Extra

Essas são as opções mais acessíveis e com as melhores características para o preço.

Espero que isso ajude! Se tiver alguma dúvida adicional, sinta-se à vontade para perguntar.


All right! Now we understand how to use the ``WebBaseLoader`` to load websites and take text from it!